In [81]:
from revChatGPT.V1 import Chatbot
import random
import json
import sqlite3

f = open("config.json")
config = json.load(f)


def get_free_chatbot(index):
    # print('free index: ', index)
    free_count = 3
    name = ""
    if index % free_count == 0:
        name = "access_token_r105"
    elif index % free_count == 1:
        name = "access_token_vic_yy"
    elif index % free_count == 2:
        name = "access_token_tony"
    print(f"free name: {name}, index: {index}", end="\r", flush=True)
    chatbot = Chatbot(config={"access_token": config[name]})
    return chatbot


def get_pay_chatbot(index):
    # print('pay index: ', index)
    pay_count = 4
    name = ""
    if index % pay_count == 0:
        name = "access_token_vic"
    elif index % pay_count == 1:
        name = "access_token_vic"
    elif index % pay_count == 2:
        name = "access_token_lab"
    elif index % pay_count == 3:
        rand_int = random.randrange(5)
        # 為了讓這隻帳號不要被跑太多次
        if rand_int == 0:
            name = "access_token_lab"
        elif rand_int == 1:
            name = "access_token_lab"
        elif rand_int == 2:
            name = "access_token_vic"
        elif rand_int == 3:
            name = "access_token_charlie"

    print(f"pay name: {name}, index: {index}", end="\r", flush=True)
    chatbot = Chatbot(config={"access_token": config[name]})
    return chatbot


import pandas as pd
import time
import os
import sys

tmp_config = {"dateset": "./data/", "output": "predicted_sql.txt"}
# if sys.argv[1] == "--dataset" and sys.argv[3] == "--output":
DATASET_SCHEMA = tmp_config["dateset"] + "tables.json"
DATASET = tmp_config["dateset"] + "dev.json"
OUTPUT_FILE = tmp_config["output"]
FILE_PATH = "my_result/resqsql/3"
BEFORE_SQL = f"{FILE_PATH}/resd_predicted_sql.txt"
SCHEMA_LINK_FILE = "my_result/resqsql/schema_link.txt"
DB_PATH = '/home/r10525068/ensemble/data/database'
# else:
#     raise Exception("Please use this format python CoT.py --dataset data/ --output predicted_sql.txt")

import re


def remove_non_english(string):
    pattern = r"^[^a-zA-Z]+|[^a-zA-Z]+$"
    cleaned_string = re.sub(pattern, "", string)
    return cleaned_string


def load_data(DATASET):
    return pd.read_json(DATASET)


def load_sql_txt(file_path):
    with open(file_path, "r") as f:
        return [line.strip() for line in f]


def find_foreign_keys_MYSQL_like(db_name):
    df = spider_foreign[spider_foreign["Database name"] == db_name]
    output = "["
    for index, row in df.iterrows():
        output += (
            row["First Table Name"]
            + "."
            + row["First Table Foreign Key"]
            + " = "
            + row["Second Table Name"]
            + "."
            + row["Second Table Foreign Key"]
            + ","
        )
    output = output[:-1] + "]"
    return output


def find_fields_MYSQL_like(db_name):
    df = spider_schema[spider_schema["Database name"] == db_name]
    df = df.groupby(" Table Name")
    output = ""
    for name, group in df:
        output += "Table " + name + ", columns = ["
        for index, row in group.iterrows():
            output += row[" Field Name"] + ","
        output = output[:-1]
        output += "]\n"
    return output

def find_cols_with_entities_by_sql(SQL):
    sql_str_list = SQL.split(' ')
    tb = None
    col = None
    cols_have_entities = []
    for sql_idx, str in enumerate(sql_str_list):
        if str.startswith('\''):
            try:
                tb_col = sql_str_list[sql_idx-2]
                if '.' in tb_col:
                    col = remove_non_english(tb_col.split('.')[1])
                    cols_have_entities.append(col)
                else:
                    col = remove_non_english(tb_col.strip(' '))
                    cols_have_entities.append(col)
            except:
                print('cannot split it:', sql_str_list[sql_idx-2])
    return cols_have_entities


def find_table_by_fields(db_name, cols_have_entities):
    df = spider_schema[spider_schema["Database name"] == db_name]
    df = df.groupby(" Table Name")
    tables = {}
    for name, group in df:
        for index, row in group.iterrows():
            if row[" Field Name"] in cols_have_entities:
                col = row[" Field Name"]
                if name not in tables:
                    tables[name] = []
                tables[name].append(col)
    return tables

def find_entities_by_table_fields(db, tables_with_cols):
    connection = sqlite3.connect(f'{DB_PATH}/{db}/{db}.sqlite')
    cursor = connection.cursor()
    cols_with_entities = {}
    for table in tables_with_cols.keys():
        for col in tables_with_cols[table]:
            res = cursor.execute(f"SELECT DISTINCT {col} FROM {table} LIMIT 10")
            value = res.fetchall()
            try:
                int(value[0][0])
            except:
                cols_with_entities[col] = [v[0] for v  in value]
    return cols_with_entities
    

def find_primary_keys_MYSQL_like(db_name):
    df = spider_primary[spider_primary["Database name"] == db_name]
    output = "["
    for index, row in df.iterrows():
        output += row["Table Name"] + "." + row["Primary Key"] + ","
    output = output[:-1]
    output += "]\n"
    return output


def creatiing_schema(DATASET_JSON):
    schema_df = pd.read_json(DATASET_JSON)
    schema_df = schema_df.drop(["column_names", "table_names"], axis=1)
    schema = []
    f_keys = []
    p_keys = []
    for index, row in schema_df.iterrows():
        tables = row["table_names_original"]
        col_names = row["column_names_original"]
        col_types = row["column_types"]
        foreign_keys = row["foreign_keys"]
        primary_keys = row["primary_keys"]
        for col, col_type in zip(col_names, col_types):
            index, col_name = col
            if index == -1:
                for table in tables:
                    schema.append([row["db_id"], table, "*", "text"])
            else:
                schema.append([row["db_id"], tables[index], col_name, col_type])
        for primary_key in primary_keys:
            index, column = col_names[primary_key]
            p_keys.append([row["db_id"], tables[index], column])
        for foreign_key in foreign_keys:
            first, second = foreign_key
            first_index, first_column = col_names[first]
            second_index, second_column = col_names[second]
            f_keys.append(
                [
                    row["db_id"],
                    tables[first_index],
                    tables[second_index],
                    first_column,
                    second_column,
                ]
            )
    spider_schema = pd.DataFrame(
        schema, columns=["Database name", " Table Name", " Field Name", " Type"]
    )
    spider_primary = pd.DataFrame(
        p_keys, columns=["Database name", "Table Name", "Primary Key"]
    )
    spider_foreign = pd.DataFrame(
        f_keys,
        columns=[
            "Database name",
            "First Table Name",
            "Second Table Name",
            "First Table Foreign Key",
            "Second Table Foreign Key",
        ],
    )
    return spider_schema, spider_primary, spider_foreign

def entities_debuger(test_sample_text, database, sql, cols_have_entities):
    instruction = """#### For the given question, use the provided cell values to fix the given SQLite SQL QUERY for any issues. If there are any problems, fix them. If there are no issues, return the SQLite SQL QUERY as is.
"""
    tables_with_cols = find_table_by_fields(database, cols_have_entities)
    cols_with_entities = find_entities_by_table_fields(database, tables_with_cols)
    print(cols_with_entities)
    fields = ""
    for col in cols_with_entities.keys():
        fields += f"column = [{col}\n"
        fields += f"cell value has [{', '.join(cols_with_entities[col])}]\n"
    # fields = find_fields_MYSQL_like(database)
    # fields += "Foreign_keys = " + find_foreign_keys_MYSQL_like(database) + "\n"
    # fields += "Primary_keys = " + find_primary_keys_MYSQL_like(database)
    prompt = (
        instruction
        + fields
        + "#### Question: "
        + test_sample_text
        + "\n#### SQLite SQL QUERY\n"
        + sql
        + "\n#### SQLite FIXED SQL QUERY\nSELECT"
    )
    return prompt


def GPT4_debug(chatbot, prompt, model="text-davinci-002-render-sha"):
    for data in chatbot.ask(prompt, conversation_id=None, parent_id=None, model=model):
        response = data["message"]
    GPT4_clear_conversations(chatbot)
    return response


def GPT4_clear_conversations(chatbot):
    chatbot.clear_conversations()


spider_schema, spider_primary, spider_foreign = creatiing_schema(DATASET_SCHEMA)
val_df = load_data(DATASET)
SQL_list = load_sql_txt(BEFORE_SQL)
schema_link_list = load_sql_txt(SCHEMA_LINK_FILE)

In [82]:
for ((index, row), SQL, schema_link) in zip(val_df.iterrows(), SQL_list, schema_link_list):
    if index  != 215: continue
    db = row['db_id']
    print('\nindex:', index)
    # 直接空白鍵切割字串，遇到雙引號、單引號，代表entities，此值的前兩個位置就是該table及欄位，取得它然後跑cursor
    
    cols_have_entities = find_cols_with_entities_by_sql(SQL)
    if len(cols_have_entities) == 0:
        # print(SQL)
        pass
    else:
        prompt = entities_debuger(row['question'], row['db_id'], SQL, cols_have_entities)
        print(prompt)
    # print(cols_with_entities)
    if index > 300: break


index: 215
{'Airline': ['United Airlines', 'US Airways', 'Delta Airlines', 'Southwest Airlines', 'American Airlines', 'Northwest Airlines', 'Continental Airlines', 'JetBlue Airways', 'Frontier Airlines', 'AirTran Airways'], 'SourceAirport': [' APG', ' ASY', ' CVO', ' ACV', ' AHD', ' AHT', ' ATO', ' ABR', ' ANV', ' MMI'], 'DestAirport': [' ASY', ' APG', ' ACV', ' CVO', ' AHT', ' AHD', ' ABR', ' ATO', ' MMI', ' ANV']}
#### For the given question, use the provided cell values to fix the given SQLite SQL QUERY for any issues. If there are any problems, fix them. If there are no issues, return the SQLite SQL QUERY as is.
column = [Airline]
cell value includes [United Airlines, US Airways, Delta Airlines, Southwest Airlines, American Airlines, Northwest Airlines, Continental Airlines, JetBlue Airways, Frontier Airlines, AirTran Airways]
column = [SourceAirport]
cell value includes [ APG,  ASY,  CVO,  ACV,  AHD,  AHT,  ATO,  ABR,  ANV,  MMI]
column = [DestAirport]
cell value includes [ ASY, 

In [35]:
import re

sql_string = "SELECT AirportName  FROM airports  WHERE AirportCode = 'AKO'"
print('\'AKO\'' in sql_string.split(' '))

True


In [11]:
res = cursor.execute("SELECT * FROM singer LIMIT 5")

In [12]:
res.fetchall()

[(1, 'Joe Sharp', 'Netherlands', 'You', '1992', 52, 'F'),
 (2, 'Timbaland', 'United States', 'Dangerous', '2008', 32, 'T'),
 (3, 'Justin Brown', 'France', 'Hey Oh', '2013', 29, 'T'),
 (4, 'Rose White', 'France', 'Sun', '2003', 41, 'F'),
 (5, 'John Nizinik', 'France', 'Gentleman', '2014', 43, 'T')]

In [24]:
sql_string = "SELECT COUNT(DISTINCT ModelId) FROM model_list JOIN car_makers ON model_list.Maker = car_makers.Id WHERE car_makers.FullName = 'American Motor Company'"

sql_str_list = sql_string.split(' ')
for index, str in enumerate(sql_str_list):
    if str.startswith('\''):
        print(str)
        try:
            tb_col = sql_str_list[index-2]
            tb = tb_col.split('.')[0]
            col = tb_col.split('.')[1]
        except:
            print(sql_str_list[index-2])

'American
car_makers.FullName


'a'

In [83]:
my_dict = {"Key1": "Value1", "KEY2": "Value2", "kEy3": "Value3"}

key_to_find = "key1"
if key_to_find.lower() in my_dict:
    print("键存在于字典中。")
else:
    print("键不存在于字典中。")


键不存在于字典中。
